<a href="https://colab.research.google.com/github/leegang/colab-notebook/blob/main/vts_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple video to translate and lipsync pipeline


Upload a video of someone speaking and get it back in another language, same voice and lip synched.

Just a fun proof of concept pipeline, will probably reduce bloat and actually implement later.

High quality reduces the gaussian blur significantly and I think it looks much better but it takes a lot longer.

Credits to:


*   https://github.com/coqui-ai/TTS
*   https://github.com/OpenTalker/video-retalking/
*   https://github.com/openai/whisper
*   https://github.com/justinjohn0306/Wav2Lip/

Notes:

- 60s is the max it can go, afaik.
- mileage may vary with voice synthesis, if it doesnt sound quite right just regenerate
- if it crashes even after freeing the memory from the other models, just restart, upload the video again and run only the lip sync part
- if you find any bugs pls dm me since i did this at 3am @yeswondwerr

## Part 1: extract, translate and generate voice

In [ ]:
# @title Dependencies

import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install TTS
!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install googletrans==4.0.0-rc1

In [ ]:
#@title Upload Video

from google.colab import files
import os
import subprocess

uploaded = None
resize_to_720p = False

def upload_video():
  global uploaded
  global video_path  # Declare video_path as global to modify it
  uploaded = files.upload()
  for filename in uploaded.keys():
    print(f'Uploaded {filename}')
    if resize_to_720p:
        filename = resize_video(filename)  # Get the name of the resized video
    video_path = filename  # Update video_path with either original or resized filename
    return filename


def resize_video(filename):
    output_filename = f"resized_{filename}"
    cmd = f"ffmpeg -i {filename} -vf scale=-1:720 {output_filename}"
    subprocess.run(cmd, shell=True)
    print(f'Resized video saved as {output_filename}')
    return output_filename

# Create a form button that calls upload_video when clicked and a checkbox for resizing
import ipywidgets as widgets
from IPython.display import display

button = widgets.Button(description="Upload Video")
checkbox = widgets.Checkbox(value=False, description='Resize to 720p (better results)')
output = widgets.Output()

def on_button_clicked(b):
  with output:
    global video_path
    global resize_to_720p
    resize_to_720p = checkbox.value
    video_path = upload_video()

button.on_click(on_button_clicked)
display(checkbox, button, output)


In [ ]:
# @title Audio extraction (24 bit) and whisper conversion
import subprocess

# Ensure video_path variable exists and is not None
if 'video_path' in globals() and video_path is not None:
    ffmpeg_command = f"ffmpeg -i '{video_path}' -acodec pcm_s24le -ar 48000 -q:a 0 -map a -y 'output_audio.wav'"
    subprocess.run(ffmpeg_command, shell=True)
else:
    print("No video uploaded. Please upload a video first.")

import whisper

model = whisper.load_model("base")
result = model.transcribe("output_audio.wav")

whisper_text = result["text"]
whisper_language = result['language']

print("Whisper text:", whisper_text)

In [ ]:
#@title Translation
# Mapping between full names and ISO 639-1 codes
language_mapping = {
    'English': 'en',
    'Spanish': 'es',
    'French': 'fr',
    'German': 'de',
    'Italian': 'it',
    'Portuguese': 'pt',
    'Polish': 'pl',
    'Turkish': 'tr',
    'Russian': 'ru',
    'Dutch': 'nl',
    'Czech': 'cs',
    'Arabic': 'ar',
    'Chinese (Simplified)': 'zh-cn'
}

# Dropdown with full names
target_language = "English" #@param ["English", "Spanish", "French", "German", "Italian", "Portuguese", "Polish", "Turkish", "Russian", "Dutch", "Czech", "Arabic", "Chinese (Simplified)"]

# Convert full name to ISO 639-1 code
target_language_code = language_mapping[target_language]

# Assume whisper_text and whisper_language are defined from previous code
from googletrans import Translator

# Initialize the translator
translator = Translator()

# Translate the text
translated_text = translator.translate(whisper_text, src=whisper_language, dest=target_language_code).text

# Output the translated text
print("Translated text:", translated_text)


In [ ]:
# @title Voice synthesis
from TTS.api import TTS
import torch
from IPython.display import Audio, display  # Import the Audio and display modules

# Initialize TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v1", gpu=True)

# Generate audio file
tts.tts_to_file(translated_text,
    speaker_wav='output_audio.wav',
    file_path="output_synth.wav",
    language=target_language_code
)

# Display audio widget to play the generated audio
audio_widget = Audio(filename="output_synth.wav", autoplay=False)
display(audio_widget)

In [ ]:
# @title Delete tts and whisper models before lip sync if you are on T4
import torch

try:
    del tts
except NameError:
    print("Voice model already deleted")

try:
    del model
except NameError:
    print("Whisper model already deleted")

torch.cuda.empty_cache()

## Part 2: generate lip synched video

### High Quality (very slow, aprox 15 min to install dependencies and 12 more for vid on T4)

In [ ]:
# @title Dependencies
%cd /content/

!git clone https://github.com/vinthony/video-retalking.git &> /dev/null

!sudo apt-get install -y libblas-dev liblapack-dev libx11-dev libopenblas-dev

!git clone https://github.com/davisking/dlib.git

!pip install basicsr==1.4.2 face-alignment==1.3.4 kornia==0.5.1 ninja==1.10.2.3 einops==0.4.1 facexlib==0.2.5 librosa==0.9.2 build

!cd dlib && python setup.py install

%cd /content/video-retalking

!mkdir ./checkpoints
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/30_net_gen.pth -O ./checkpoints/30_net_gen.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/BFM.zip -O ./checkpoints/BFM.zip
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/DNet.pt -O ./checkpoints/DNet.pt
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/ENet.pth -O ./checkpoints/ENet.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/expression.mat -O ./checkpoints/expression.mat
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/face3d_pretrain_epoch_20.pth -O ./checkpoints/face3d_pretrain_epoch_20.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/GFPGANv1.3.pth -O ./checkpoints/GFPGANv1.3.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/GPEN-BFR-512.pth -O ./checkpoints/GPEN-BFR-512.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/LNet.pth -O ./checkpoints/LNet.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/ParseNet-latest.pth -O ./checkpoints/ParseNet-latest.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/RetinaFace-R50.pth -O ./checkpoints/RetinaFace-R50.pth
!wget https://github.com/vinthony/video-retalking/releases/download/v0.0.1/shape_predictor_68_face_landmarks.dat -O ./checkpoints/shape_predictor_68_face_landmarks.dat
!unzip -d ./checkpoints/BFM ./checkpoints/BFM.zip

In [ ]:
# @title Generate video

%cd /content/video-retalking

video_path_fix = f"'../{video_path}'"

!python inference.py \
  --face $video_path_fix \
  --audio "/content/output_synth.wav" \
  --outfile '/content/output_high_qual.mp4'

### Normal quality (around 5 min on T4)

In [ ]:
# @title Dependencies
%cd /content/

!git clone https://github.com/justinjohn0306/Wav2Lip
!cd Wav2Lip && pip install -r requirements_colab.txt

%cd /content/Wav2Lip

!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "face_detection/detection/sfd/s3fd.pth"
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip.pth' -O 'checkpoints/wav2lip.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/resnet50.pth' -O 'checkpoints/resnet50.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth' -O 'checkpoints/mobilenet.pth'

!pip install batch-face

In [ ]:
# @title Generate video

%cd /content/Wav2Lip

#This is the detection box padding, if you see it doesnt sit quite right, just adjust the values a bit. Usually the bottom one is the biggest issue
pad_top =  0
pad_bottom =  15
pad_left =  0
pad_right =  0
rescaleFactor =  1

video_path_fix = f"'../{video_path}'"

!python inference.py --checkpoint_path 'checkpoints/wav2lip_gan.pth' --face $video_path_fix --audio "/content/output_synth.wav" --pads $pad_top $pad_bottom $pad_left $pad_right --resize_factor $rescaleFactor --nosmooth --outfile '/content/output_video.mp4'


# End

In [ ]:
# @title Run this cell to get the video/s and download links
from google.colab import files
from IPython.core.display import display, HTML
import ipywidgets as widgets
import base64
import os

# List of video paths to check
video_paths = ["/content/output_video.mp4", "/content/output_high_qual.mp4"]

# Download function
def download_video(b):
    files.download(b.video_path)

# Create button widget for download
download_buttons = []

# Layout definition for button
button_layout = widgets.Layout(width='250px')

# Loop through each video path to check for existence and display
for video_path in video_paths:
    if os.path.exists(video_path):
        # Encode video to base64
        with open(video_path, "rb") as video_file:
            video_base64 = base64.b64encode(video_file.read()).decode()

        # Create HTML widget for video
        video_html = HTML(data=f"""
        <video width=400 controls>
            <source src="data:video/mp4;base64,{video_base64}" type="video/mp4" />
        </video>
        """)

        # Create button widget for download and link to the video path
        download_button = widgets.Button(description=f"Download {os.path.basename(video_path)}",
                                         layout=button_layout)
        download_button.video_path = video_path
        download_button.on_click(download_video)
        download_buttons.append(download_button)

        # Display widgets
        display(video_html)
        display(download_button)
